In [ ]:
%cd C:\Users\AnnNg\OneDrive\Desktop\IntGfxENV

In [1]:
from setup import make
from Base.TLMN._env import bot_lv0
env = make("TLMN")

ModuleNotFoundError: No module named 'PIL'

In [ ]:
env.render(Agent="human", per_data=[0], level=1, max_temp_frame=100)